In [2]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Key
from config import api_key
c = Census(api_key, year=2016)

In [3]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E","B25058_001E","B25077_001E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B25058_001E": "average rent",
                                      "B25077_001E": "Median Home Value",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zipcode", "Name","Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate","average rent",]]
# Filter for CA 90001 - 96162
census_pd = census_pd[(census_pd['Zipcode'] >= str(90001)) & (census_pd['Zipcode'] <= str(96162))]
# Visualize
print(len(census_pd))
census_pd.head()

1763


,Zipcode,Name,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,average rent
29879,90001,ZCTA5 90001,57942.0,28.0,34323.0,11294.0,18902.0,32.622277,915.0
29880,90002,ZCTA5 90002,51826.0,27.5,32520.0,11212.0,18302.0,35.314321,902.0
29881,90003,ZCTA5 90003,70208.0,28.0,31878.0,10611.0,24556.0,34.976071,958.0
29882,90004,ZCTA5 90004,63095.0,35.5,43180.0,29194.0,12919.0,20.475473,1044.0
29883,90005,ZCTA5 90005,39338.0,33.9,31485.0,20265.0,11520.0,29.284661,943.0


In [4]:
#save as csv
census_pd.to_csv("census_data.csv",index = False)